In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
import tushare as ts
import time
import json
with open('token.json') as token_file:
    token = json.load(token_file)['token']

pro = ts.pro_api(token)
def get_daily(ts_code='', trade_date='', start_date='', end_date=''):
    for _ in range(3):
        try:
            if trade_date:
                df = pro.daily(ts_code=ts_code, trade_date=trade_date)
            else:
                df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
        except:
            time.sleep(1)
        else:
            return df

In [4]:
# 不要开梯子获取数据，返回空
target_df = pro.daily(exchange='SSE',ts_code='600812.SH',start_date='20200101', end_date='20220210')
target_cols = list(target_df.columns)
target_cols.remove('close')
target_cols.remove('ts_code')
target_cols.remove('trade_date')
X_stocks = target_df[target_cols]
y_stocks = target_df.close

In [5]:
split_ratio = 0.2
split_idx = -int(len(target_df)*split_ratio)

train_x, train_y = X_stocks[:split_idx], y_stocks[:split_idx]
test_x, test_y = X_stocks[split_idx:], y_stocks[split_idx:]

train_x = torch.from_numpy(train_x.values)
train_y = torch.from_numpy(train_y.values)
test_x = torch.from_numpy(test_x.values)
test_y = torch.from_numpy(test_y.values)

In [18]:
train_x.shape

torch.Size([400, 8])

In [8]:
class LSTMModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.batch_size=32
        self.input_shape = 8
        self.hidden_size = 50
        self.drop_out = 0.2
        self.output_size = 1
        self.lstm = nn.LSTM(input_size=self.input_shape, hidden_size=self.hidden_size)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        self.hidden_cell = (torch.zeros(1,1, self.hidden_size),
                            torch.zeros(1,1, self.hidden_size))

    def forward(self,input_seq):
        lstm_out,self.hidden_cell  = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        out = self.fc(lstm_out.view(len(input_seq)), -1)
        return out

In [9]:
def training_loop(n_epochs, optimiser, model, loss_fn, X_train,  X_val, y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        output_train = model(X_train) # forwards pass
        loss_train = loss_fn(output_train, y_train) # calculate loss
        output_val = model(X_val)
        loss_val = loss_fn(output_val, y_val)

        optimiser.zero_grad() # set gradients to zero
        loss_train.backward() # backwards pass
        optimiser.step() # update model parameters
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [10]:
model = LSTMModule()
loss_func = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.1)
print(model)

LSTMModule(
  (lstm): LSTM(8, 50)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)


In [11]:
for name, param in model.named_parameters():
    print(name, param.type(), param.dtype, param.shape)

lstm.weight_ih_l0 torch.DoubleTensor torch.float64 torch.Size([200, 8])
lstm.weight_hh_l0 torch.DoubleTensor torch.float64 torch.Size([200, 50])
lstm.bias_ih_l0 torch.DoubleTensor torch.float64 torch.Size([200])
lstm.bias_hh_l0 torch.DoubleTensor torch.float64 torch.Size([200])
fc.weight torch.DoubleTensor torch.float64 torch.Size([1, 50])
fc.bias torch.DoubleTensor torch.float64 torch.Size([1])


In [12]:
# optimiser = optim.SGD(seq_model.parameters(), lr=1e-3)
training_loop(
    n_epochs = 50,
    optimiser = optimiser,
    model = model,
    loss_fn = loss_func,
    X_train = train_x,
    X_val = test_x,
    y_train = train_y,
    y_val = test_y)

RuntimeError: shape '[400]' is invalid for input of size 20000

In [ ]:
# ## Prediction
# def predict(pred_data, model):
#     output = torch.tensor([])
#     model.eval()
#     with torch.no_grad():
#         for  in pred_data:
#             y = model()
#             output = torch.cat((output, y),0)
#     return output

In [ ]:
# predict(x_test,model).numpy()